# Lemon Game Judge

This project is based on the famous game called "The Lemon in the Spoon Race" in which we need to hold the spoon straight in order to keep the lemon from falling off. Detect and track the lemon on the spoon.

# Algorithm Used

Project implements following algorithm:-

1). Get the video.

2). Divide the video into frames and for each frame do step 3-10.

3). Convert each frame from BGR to HSV.

4). Mask those objects in the HSV image which are in HSV threshold defined.

5). Remove noise surrounding the object tracked by morphological algorithm called opening.

6). Remove noise within the object tracked using closing.

7). Retrieve number of pixels of finally masked image(img_mask3) whose value if non-zero then we have lemon in the frame, else lemon has fallen.

8). Get the contours from the frame.

9). Find the largest contour in the mask, then use it to compute the minimum enclosing circle and centroid.

10). Display the circle and it's centre on the video frame.

11). Close all windows.


# Importing required packages


We will be requiring numpy array and opencv.

In [1]:
import cv2
import numpy as np

# HSV color range

In [2]:
# Define HSV range of lemon color
HSV_Lower = np.array([25,100,100])
HSV_Upper = np.array([65,255,255])

In [3]:
# Two kernels to perform morphological operation on video frames( i.e., MASKING)
window1 = np.ones((5,5))
window2 = np.ones((20,20))

# Iterating through the video frames

Getting the frames from the video and apply mask on each frame and check the contours for circular shape.

In [4]:
def get_frames(vid):

    # Video capture object to acquire webcam feed
    cap = cv2.VideoCapture(vid)
    
    # Will return true if capture is acquired successfully
    print(cap.isOpened())
    
    currentFrame = 0

    # Capturing first frame
    ret, frame = cap.read()
    
    # Capturing frames till cap.read() returns true
    while(ret):
        
        # Capture frame-by-frame
        ret, frame = cap.read()

        # Resize the frame
        frame = cv2.resize(frame, (700, 700))

        # Convert BGR to HSV
        imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)        
 
        # Masking object in HSV lemon color range
        img_mask1 = cv2.inRange(imgHSV, HSV_Lower, HSV_Upper)       
        
        # Removing noise surrounding the object tracked
        img_mask2 = cv2.morphologyEx(img_mask1, cv2.MORPH_OPEN, window1)
        
        # Removing noise within the object tracked
        img_mask3 = cv2.morphologyEx(img_mask2, cv2.MORPH_CLOSE, window2)
             
        
        # Retrieve number of pixels of finally masked image
        if  cv2.countNonZero(img_mask3) == 0 :
                print("OOPS!! LEMON DROPPED")
                break
        else:
            if currentFrame==0 :
                print("GOOD BALANCE!!")
                
            
        # To get contours or boundaries of the tracked objects 
        _, contours, hierarchy = cv2.findContours(img_mask3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
            
        if len(contours) > 0:

            # Getting largest contour
            c = max(contours, key = cv2.contourArea)
            
            # Getting minimum enclosing circle
            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            
            # Getting centre
            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

            # Only proceed if the radius meets a minimum size
            if radius > 4:
                
                # Draw the circle and centroid on the frame,
                cv2.circle(frame, (int(x), int(y)), int(radius),
                    (0, 255, 255), 2)
                
                # Update the list of tracked points
                cv2.circle(frame, center, 5, (0, 0, 255), -1) 
                
        # Display the masked images on window
        cv2.imshow("img_mask3",img_mask3)
        cv2.imshow("cam",frame)
        cv2.waitKey(10)          

        # To stop duplicate images
        currentFrame += 1 
        
    # When everything done, release the capture and close all windows
    cap.release
    cv2.destroyAllWindows()
    

# Trying the video

In [5]:
vid  = "haha.mp4"
get_frames(vid)

True
GOOD BALANCE!!
OOPS!! LEMON DROPPED
